# Calculating significance of Census estimates
References:
* [Using American Community Survey Estimates and Margins of Error presententation](https://www.census.gov/content/dam/Census/programs-surveys/acs/guidance/training-presentations/20170419_MOE.pdf) from the US Census Bureau
* [Downloaded Table B02001 - RACE for 2011 & 2016 5 year estimates for Erie County census tracts](https://factfinder.census.gov/faces/nav/jsf/pages/download_center.xhtml#none)

In [1]:
erie_race_data_16 = "data/ACS_16_5YR_B02001_with_ann.csv"
erie_race_data_11 = "data/ACS_11_5YR_B02001_with_ann.csv"

In [2]:
import agate
import decimal
import math

specified_types = {
    'GEO.id': agate.Text(),
    'GEO.id2': agate.Text()
}

race_table_16 = agate.Table.from_csv(erie_race_data_16, column_types=specified_types)
race_table_11 = agate.Table.from_csv(erie_race_data_11, column_types=specified_types)

In [5]:
race_table_11.print_table()

| GEO.id               | GEO.id2     | GEO.display-label    | HD01_VD01 | HD02_VD01 | HD01_VD02 | ... |
| -------------------- | ----------- | -------------------- | --------- | --------- | --------- | --- |
| 1400000US36029000110 | 36029000110 | Census Tract 1.10... |     2,598 |       354 |     2,375 | ... |
| 1400000US36029000200 | 36029000200 | Census Tract 2, E... |     4,143 |       349 |     3,520 | ... |
| 1400000US36029000500 | 36029000500 | Census Tract 5, E... |     2,219 |       383 |     2,189 | ... |
| 1400000US36029000600 | 36029000600 | Census Tract 6, E... |     4,829 |       542 |     4,230 | ... |
| 1400000US36029000700 | 36029000700 | Census Tract 7, E... |     3,816 |       327 |     3,750 | ... |
| 1400000US36029000800 | 36029000800 | Census Tract 8, E... |     5,298 |       439 |     4,974 | ... |
| 1400000US36029000900 | 36029000900 | Census Tract 9, E... |     2,223 |       259 |     2,142 | ... |
| 1400000US36029001000 | 36029001000 | Census Tract 10, ... |   

## Calculate whether one variable is significant

In [6]:
class Significant90CI(agate.Computation):
    """
    Calculate whether a census variable is statistically significant given MOE
    """
    def __init__(self, column_value, column_moe):
        self._column_value = column_value
        self._column_moe = column_moe

    def get_computed_data_type(self, table):
        """
        The return value is a numerical distance.
        """
        return agate.Number()

    def validate(self, table):
        """
        Verify the columns are numbers.
        """
        value_column = table.columns[self._column_value]
        moe_column = table.columns[self._column_moe]

        if not isinstance(value_column.data_type, agate.Number) or not isinstance(moe_column.data_type, agate.Number):
            raise agate.DataTypeError('Can only be applied to Number data.')

    def run(self, table):
        """
        Returning value
        """
        new_column = []

        for row in table.rows:
            val = row[self._column_value]
            moe = row[self._column_moe]
            sig = abs(val/(moe/decimal.Decimal(1.645)))
            new_column.append(sig)

        return new_column

In [7]:
white_percent = 'HD01_VD01'
white_MOE = 'HD02_VD01'
black_percent = 'HD01_VD03'
black_MOE = 'HD02_VD03'

In [9]:
sig_race_16 = race_table_16.compute([
    ('sig_white', Significant90CI(white_percent, white_MOE)),
    ('sig_black', Significant90CI(black_percent, black_MOE))
])

In [13]:
sig_race_16.select(['GEO.id2', white_percent, white_MOE,'sig_white', 'sig_black']).print_table()

| GEO.id2     | HD01_VD01 | HD02_VD01 | sig_white | sig_black |
| ----------- | --------- | --------- | --------- | --------- |
| 36029000110 |     2,934 |       348 |   13.869… |    2.017… |
| 36029000200 |     4,146 |       441 |   15.465… |    2.393… |
| 36029000500 |     1,971 |       307 |   10.561… |    1.567… |
| 36029000600 |     4,686 |       450 |   17.130… |    1.578… |
| 36029000700 |     3,659 |       284 |   21.194… |    1.645… |
| 36029000800 |     4,603 |       509 |   14.876… |    1.807… |
| 36029000900 |     2,485 |       217 |   18.838… |    1.387… |
| 36029001000 |     5,679 |       630 |   14.829… |    2.967… |
| 36029001100 |     2,843 |       277 |   16.884… |    2.135… |
| 36029001402 |     2,743 |       297 |   15.193… |   13.116… |
| 36029001500 |     1,428 |       212 |   11.080… |    9.590… |
| 36029001600 |     1,993 |       293 |   11.189… |    8.773… |
| 36029001700 |     1,730 |       291 |    9.780… |    3.205… |
| 36029001900 |     3,052 |       284 | 

In [14]:
print(len(sig_race_16.rows))

237


In [15]:
sig_race_90_16 = sig_race_16.where(lambda row: row['sig_white'] > decimal.Decimal(1.645) and row['sig_black'] > decimal.Decimal(1.645))

In [16]:
print(len(sig_race_90_16))

151


## Calculate Margin of Error given two estimates & margin of error

In [25]:
joined = race_table_11.join(race_table_16,'GEO.id2', 'GEO.id2',full_outer=True)

/Users/lheinle/anaconda/lib/python3.5/site-packages/agate/utils.py:291: DuplicateColumnWarning: Column name "GEO.id2" already exists in Table. Column will be renamed to "GEO.id2_2".


In [41]:
def calculate_moe(row):
    factor = 1.645
    moe_11 = float(row[white_MOE])
    moe_16 = float(row['HD02_VD012'])
    return math.sqrt(math.pow((moe_11/factor), 2)+math.pow((moe_16/factor), 2))*factor
    return row['price'].quantize(Decimal('0.01'))

joined_change = joined.compute([
    ('change1116', agate.Change(white_percent,'HD01_VD012')),
    ('moe1116', agate.Formula(agate.Number(), calculate_moe)),
])
joined_2 = joined_change.compute([
    ('sig_1116', Significant90CI('change1116', 'moe1116')),
])

In [42]:
joined_2.select(['GEO.id2', white_percent,'HD01_VD012','change1116', 'moe1116','sig_1116']).print_table()

| GEO.id2     | HD01_VD01 | HD01_VD012 | change1116 |  moe1116 | sig_1116 |
| ----------- | --------- | ---------- | ---------- | -------- | -------- |
| 36029000110 |     2,598 |      2,934 |        336 | 496.407… |   1.113… |
| 36029000200 |     4,143 |      4,146 |          3 | 562.390… |   0.009… |
| 36029000500 |     2,219 |      1,971 |       -248 | 490.854… |   0.831… |
| 36029000600 |     4,829 |      4,686 |       -143 | 704.460… |   0.334… |
| 36029000700 |     3,816 |      3,659 |       -157 | 433.111… |   0.596… |
| 36029000800 |     5,298 |      4,603 |       -695 | 672.162… |   1.701… |
| 36029000900 |     2,223 |      2,485 |        262 | 337.891… |   1.276… |
| 36029001000 |     5,872 |      5,679 |       -193 | 833.676… |   0.381… |
| 36029001100 |     2,643 |      2,843 |        200 | 376.502… |   0.874… |
| 36029001402 |     3,735 |      2,743 |       -992 | 541.681… |   3.013… |
| 36029001500 |     1,400 |      1,428 |         28 | 336.251… |   0.137… |
| 3602900160

In [39]:
sig_year_90_1116 = joined_2.where(lambda row: row['sig_1116'] > decimal.Decimal(1.645))

In [40]:
print(len(joined_2.rows))
print(len(sig_year_90_1116.rows))

237
49
